# 01 Tests on Reinforcement Learning

## Intro
In this notebook I am going to test wich strategy is better to train a RL agent in a very-limited environment such as the tic-tac-toe game.


## RL algorithm
The implemented algorithm is the [Q-Learning algorithm](https://en.wikipedia.org/wiki/Q-learning#Algorithm), that is able to find the optimal policy $\pi^*$.

$Q$ is a fuction that given a state and an action, returns a number that means the *quality*, so the best move for a state is the one that maximizes the expected value of the total reward over all successive steps. In this case, winning the game.

The easiest way to implement the function $Q$ is as a matrix ($State \times Action$). And if the transition probability matrix is not known, then we have to sample from the environment by making the agent to play. This way is posible to calculate $Q$ iteratively.

\begin{equation*}
Q^{new} (s_{t},a_{t}) \leftarrow (1-\alpha) \cdot \underbrace{Q(s_{t},a_{t})}_{\text{old value}} + \underbrace{\alpha}_{\text{learning rate}} \cdot  \overbrace{\bigg( \underbrace{r_{t}}_{\text{reward}} + \underbrace{\gamma}_{\text{discount factor}} \cdot \underbrace{\max_{a}Q(s_{t+1}, a)}_{\text{estimate of optimal future value}} \bigg) }^{\text{learned value}}
\end{equation*}

Where:
* $r_{t}$ is the reward observed for the current state * $s_t$
* $\alpha \in [0,1]$ is the learning rate, which represents the importance between previous experiences and the current one.
* $\gamma \in [0,1]$ is the discount factor, which represents the difference in importance between future rewards and present rewards.

In [14]:
import itertools

from players.minimax import Minimax
from players.qlearner import QLearner
from players.random import Random
from utils import train_player_seconds, test_players, train_player_games
from datetime import datetime

---
## Teachers & Learners

### Random teacher
The random teacher chooses a random move each turn. It does not aim to win or lose but it plays very fast.

### Minimax teacher
The minimax teacher aims to win. This agent is optimal in the sense that it can only win or draw a game. It is slow.

### RL teacher
What if I make a RL agent to play against itself? Will it learn? On each game it learns to play as player 1 and player 2.

In [15]:
teachers = {
    'random': Random(2),
    'minimax': Minimax(2)
}

In [16]:
learners_seconds = {
    'random': {
        'agent': QLearner(1),
        'teacher': teachers['random'],
        'results': {},
        'train_func': lambda board,learner,teacher, game, checkpoint: learner._train_1_game(0.1, lambda x: 1-(x/10), board, teacher, game, checkpoint)
    },
    'minimax': {
        'agent': QLearner(1),
        'teacher': teachers['minimax'],
        'results': {},
        'train_func': lambda board,learner,teacher, game, checkpoint: learner._train_1_game(0.1, lambda x: 1-(x/10), board, teacher, game, checkpoint)
    },
    'rl': {
        'agent': QLearner(1),
        'teacher': None,  # Itself, referenced in future
        'results': {},
        'train_func': lambda board, learner, teacher, game, checkpoint: learner._autotrain_1_game(0.1, lambda x: 1-(x/10), board, game, checkpoint)
    },
}

learners_seconds['rl']['teacher'] = learners_seconds['rl']['agent']

---
## Experiments (limited playing time)
I am going to train three RL agents with the Q-learning algorithm. To make the experiment fair, I will limit the resources the agents have by setting a fixed trainning time. This way, if an agent has a better but costly in (CPU operations) teacher, will play less games than other with a worse but faster teacher.

In [17]:
TRAIN_SECONDS = 60  # Time for training (sg)

In [18]:
ckpt_seconds = 10

for rl_name, rl in learners_seconds.items():
    rl['seconds'] = TRAIN_SECONDS
    print("Training rl ({}) for {} seconds".format(rl_name, TRAIN_SECONDS))
    rl['games'] = train_player_seconds(
        learner=rl['agent'],
        teacher=rl['teacher'],
        train_func=rl['train_func'],
        seconds=TRAIN_SECONDS,
        checkpoint=ckpt_seconds
    )
    print("\t {} games played\n".format(rl['games']))

Training rl (random) for 60 seconds
	 121 games played

Training rl (minimax) for 60 seconds
	 92 games played

Training rl (rl) for 60 seconds
	 75 games played



---
## Metrics
To measure the actual performance of each agent, I will make them play against several opponents.

In [19]:
TEST_N_GAMES = 100  # Games to test

### Optimal (minimax)
If the agent is good enough, when faced against this oponent, no one will win and all the games will be draws.

In [20]:
def print_learners_vs_minimax(learners, n_games):
    for rl_name, rl in learners.items():
        print("Testing rl ({}) VS minimax ({} games)".format(rl_name, n_games))
        print("\tAs player 1")
        results = test_players(rl['agent'], teachers['minimax'], n_games)
        rl['results']['vs_minimax (p1)'] = {
            'wins': results[1],
            'loses':  results[2],
            'draws':  results[-1]
        }
        print(rl['results']['vs_minimax (p1)'])
        print("\tAs player 2")
        results = test_players(teachers['minimax'], rl['agent'], n_games)
        rl['results']['vs_minimax (p2)'] = {
            'wins': results[2],
            'loses':  results[1],
            'draws':  results[-1]
        }
        print(rl['results']['vs_minimax (p2)'])
        print('\n')

In [21]:
print_learners_vs_minimax(learners_seconds, TEST_N_GAMES)

Testing rl (random) VS minimax (100 games)
	As player 1
{'wins': 0, 'loses': 89, 'draws': 11}
	As player 2
{'wins': 0, 'loses': 96, 'draws': 4}


Testing rl (minimax) VS minimax (100 games)
	As player 1
{'wins': 0, 'loses': 69, 'draws': 31}
	As player 2
{'wins': 0, 'loses': 95, 'draws': 5}


Testing rl (rl) VS minimax (100 games)
	As player 1
{'wins': 0, 'loses': 92, 'draws': 8}
	As player 2
{'wins': 0, 'loses': 98, 'draws': 2}




### Against other RL agent
However, what the agent has learned could be a non-optimal policy and this way I could choose which one is the best. There can only be one.

In [22]:
def print_learners_matches(learners, n_games):
    for (rl1_name, rl1), (rl2_name,rl2) in itertools.combinations_with_replacement(learners.items(),2):
        print("Testing rl ({}) VS rl({}) ({} games)".format(rl1_name, rl2_name, n_games))
        print("\tAs player 1")
        results = test_players(rl1['agent'], rl2['agent'], n_games)
        rl['results']['vs_{} (p1)'.format(rl2_name)] = {
            'wins': results[1],
            'loses':  results[2],
            'draws':  results[-1]
        }
        print(rl['results']['vs_{} (p1)'.format(rl2_name)])
        print("\tAs player 2")
        results = test_players(rl2['agent'], rl1['agent'], n_games)
        rl['results']['vs_{} (p2)'.format(rl2_name)] = {
            'wins': results[2],
            'loses':  results[1],
            'draws':  results[-1]
        }
        print(rl['results']['vs_{} (p2)'.format(rl2_name)])
        print('\n')

In [23]:
print_learners_matches(learners_seconds, TEST_N_GAMES)

Testing rl (random) VS rl(random) (100 games)
	As player 1
{'wins': 0, 'loses': 100, 'draws': 0}
	As player 2
{'wins': 100, 'loses': 0, 'draws': 0}


Testing rl (random) VS rl(minimax) (100 games)
	As player 1
{'wins': 65, 'loses': 26, 'draws': 9}
	As player 2
{'wins': 36, 'loses': 49, 'draws': 15}


Testing rl (random) VS rl(rl) (100 games)
	As player 1
{'wins': 74, 'loses': 16, 'draws': 10}
	As player 2
{'wins': 27, 'loses': 62, 'draws': 11}


Testing rl (minimax) VS rl(minimax) (100 games)
	As player 1
{'wins': 0, 'loses': 100, 'draws': 0}
	As player 2
{'wins': 100, 'loses': 0, 'draws': 0}


Testing rl (minimax) VS rl(rl) (100 games)
	As player 1
{'wins': 46, 'loses': 38, 'draws': 16}
	As player 2
{'wins': 35, 'loses': 63, 'draws': 2}


Testing rl (rl) VS rl(rl) (100 games)
	As player 1
{'wins': 0, 'loses': 100, 'draws': 0}
	As player 2
{'wins': 100, 'loses': 0, 'draws': 0}




## Out of scope
This notebook does not include how to find the best hyperparameters, it's just a glimpse on how to explore the search space and proving that in order to learn how to win, the agent must win during the training.

---
---

## Experiments (limited number of games)

In [27]:
TRAIN_GAMES = 10  # Number of games for training

In [31]:
learners_games = {
    'random': {
        'agent': QLearner(1),
        'teacher': teachers['random'],
        'results': {},
        'train_func': lambda board,learner,teacher,games, checkpoint: learner._train_1_game(0.1, lambda x: 1-(x/10), board, teacher, games, checkpoint)
    },
    'minimax': {
        'agent': QLearner(1),
        'teacher': teachers['minimax'],
        'results': {},
        'train_func': lambda board,learner,teacher, games, checkpoint: learner._train_1_game(0.1, lambda x: 1-(x/10), board, teacher, games, checkpoint)
    },
    'rl': {
        'agent': QLearner(1),
        'teacher': None,  # Itself, referenced in future
        'results': {},
        'train_func': lambda board, learner, games, checkpoint: learner._autotrain_1_game(0.1, lambda x: 1-(x/10), board, games, checkpoint)
    },
}

learners_games['rl']['teacher'] = learners_seconds['rl']['agent']

In [ ]:
ckpt_games = 10

for rl_name, rl in learners_games.items():
    rl['games'] = TRAIN_GAMES
    print("Training rl ({}) in {} games".format(rl_name, TRAIN_GAMES))
    rl['games'] = train_player_games(
        learner=rl['agent'],
        teacher=rl['teacher'],
        train_func=rl['train_func'],
        games=TRAIN_GAMES,
        checkpoint = ckpt_games
    )
    print("\t {} seconds played\n".format(rl['games']))

---
## Metrics

In [ ]:
TEST_N_GAMES = 100  # Games to test

### Against Minimax

In [ ]:
print_learners_vs_minimax(learners_games, TEST_N_GAMES)

### Against other RL agent

In [ ]:
print_learners_matches(learners_games, TEST_N_GAMES)

---
---
## Test the bests

In [30]:
learners_best = {
    'seconds_rl': learners_seconds['rl'],
    'games_minimax': learners_games['minimax']
}

In [ ]:
print_learners_matches(learners_best, TEST_N_GAMES)

## Conclusion
These are the knowledge extracted from the experiment.


### On learning efficiency
The TicTacToe game is an easy environment for an agent as the number of different states is finite and small in contrast with a real-world problem.

In order to overcome the fact that the actual computing power allows brute-force searching strategies thus making the game a solved one, some kind of restrictions have to be set. These constraints delimit a tradeoff between the strength (quality of the actions) of the opponent the agent will have to face and the number of games it is able to play (pairs state-action in the Q function) and enables the comparison of different exploration/learning strategies.

#### Time limited
When the time to learn is fixed, having a quick response from the opponent allows to play more games. More games equals more knowledge about the game. This way, a random teacher overpowers the ability to train a RL agent of a minimax (optimal move) teacher. Although this randomly explored knowledge may not be of the best quality, it could be enough to explore all the states and therefore, know all the game.


#### Games limited
When the limitation is on the number of games to play, having an optimal player as a teacher is better than exploring randomly the game.


#### Minimax teacher
The problem with training against the minimax teacher is that the agent won't be able to learn how to win (strategy) as the minimax algorithm never loses, it wins or forces draws. So, the strategy the agent learns will be conservative as some states have not be explored. It can be seen in the game-limited experiment, the rl-minimax agent performs beter than the rl-rl agent against a minimax player but, between them the rl-rl wins.


#### auto-double learning
When the agent trains against itself, it is able to learn as player 1 and player 2 in the same game. This speeds-up the process and allows the agent to selectively (not randomly or optimally) explore the game graph. The performance is good in both cases, time and number-of-games-games constrained, as it is as good or even better than other agents. It generalizes very well. It is like having a teacher with the speed similar to a random one and the quality that matches the optimal one as more games are played.

*Note: Double learning can be done because an agent playing as player 1 has a completely different set of states as the same playing as player 2. They are 2 completely different agents.*


### Future experiments
Some ideas for the future.

#### Starting point
It will be very interesting to explore how to embed past experiences and knowledge from the environment into the agent (i.e. human experience) to have a starting point to start the training and avoid a cold start.